<a href="https://colab.research.google.com/github/xsuryanshx/DSML-Projects/blob/main/sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from keras.utils.np_utils import to_categorical
import re

In [14]:
data = pd.read_csv('/content/gdrive/MyDrive/ML stuff/Dataset.csv')
pd.set_option('display.max_colwidth',-1)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,text,stars,sentiment
0,the pizza was okay not the best ive had i prefer biaggios on flamingo fort apache the chef there can make a much better ny style pizza the pizzeria cosmo was over priced for the quality and lack of personality in the food biaggios is a much better pick if youre going for italian family owned home made recipes people that actually care if you like their food you dont get that at a pizzeria in a casino i dont care what you say,2,0
1,i love this place my fiance and i go here atleast once a week the portions are huge food is amazing i love their carne asada they have great lunch specials leticia is super nice and cares about what you think of her restaurant you have to try their cheese enchiladas too the sauce is different and amazing,5,1
2,terrible dry corn bread rib tips were all fat and mushy and had no flavor if you want bbq in this neighborhood go to john mulls roadkill grill trust me,1,0
3,back in 20052007 this place was my favorite thai place ever id go here alllll the time i never had any complaints once they started to get more known and got busy their service started to suck and their portion sizes got cut in half i have a huge problem with paying more for way less food the last time i went there i had the pork pad se ew and it tasted good but i finished my plate and was still hungry i used to know the manager here and she would greet me with a hello melissa nice to see you again diet coke pad thai or pad se ew now a days i know she still knows me but she disregards my presence also i had asked her what was up with the new portion sizes and she had no answer for me great food but not worth the money i havent been back in over a year because i refuse to pay 1015 for dinner and still be hungry after sorry pinkaow you are not what you used to be,2,0
4,delicious healthy food the steak is amazing fish and pork are awesome too service is above and beyond not a bad thing to say about this place worth every penny,5,1


In [13]:
len(data)

10000

In [33]:
tokenizer = Tokenizer()
corpus = []
for i in data['text']:
  corpus.append(i)
tokenizer.fit_on_texts(corpus)
print(tokenizer.word_index)
total_words = len(tokenizer.word_index)+1
print(total_words)

In [57]:
input_sequence = []
n = 0
for line in corpus:
  sequences = tokenizer.texts_to_sequences([line])[0]
  sequences.append(data['stars'][n])
  input_sequence.append(sequences)
  n +=1
max_len_seq = max(len(x)for x in input_sequence)
print(max_len_seq)
x = pad_sequences(input_sequence, maxlen=max_len_seq)
y = data['sentiment']

968


In [64]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2)

In [73]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(total_words,32,input_length=max_len_seq),
              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
              tf.keras.layers.Dense(1, activation='sigmoid')])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

history = model.fit(xtrain, ytrain, epochs=5, batch_size=128, verbose=1)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 968, 32)           1007808   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 64)                16640     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 1,024,513
Trainable params: 1,024,513
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
63/63 [==============================] - 64s 1s/step - loss: 0.5756 - accuracy: 0.6779
Epoch 2/5
63/63 [==============================] - 65s 1s/step - loss: 0.2338 - accuracy: 0.9312
Epoch 3/5
63/63 [==============================] - 65s 1s/step - loss: 0.0617 - accuracy: 0.9908
Epoch 4/5
63/63 [==============================] - 64s 1s/step - loss: 0.0190 - acc

In [77]:
score,acc = model.evaluate(xtest,ytest,batch_size=32)
print(acc)
print(score)

63/63 [==============================] - 6s 90ms/step - loss: 0.0279 - accuracy: 0.9935
0.9934999942779541
0.02794794552028179
